# NECモバイルバックエンド基盤 Jupyter Notebook Sample

REST API 経由でNECモバイルバックエンド基盤(以下BaaS)サーバに対してオブジェクト検索を行い、取得したデータを簡易的にプロットするサンプルです。

## 接続先BaaSサーバ情報を設定

~/.baas/python ディレクトリに python_config.yaml ファイルを作成し、BaaSサーバの接続情報を記述してください。

記述方法の詳細は、[Python SDK : 初期化](https://nec-baas.github.io/baas-manual/latest/developer/ja/python/initialize.html) ⇒ 「BaaSサーバ接続先自動設定」を参照してください。

設定ファイルを使用しない場合は、以下に直接設定情報を記述してください。


In [ ]:
config = None

# 設定情報を直接記述する場合は、以下をコメントインして設定を記述してください。
"""
config = {
    "baseUrl": "https://baas.example.com/api",
    "tenantId": "tenant1",
    "appId": "",
    "appKey": ""
    #"proxy": {
    #    "http": "proxy.example.com:8080",
    #    "https": "proxy.example.com:8080"
    #}
}
"""

## オブジェクト検索
necbaas モジュールを使い REST API 経由でオブジェクトデータを検索します。  

本サンプルでは "Sample_WeatherData" バケット内のオブジェクトを、"dateTime" フィールドの値について範囲指定で取得します。  
検索に用いるパラメータは以下の通り。
* where ： 検索条件。MongoDBのクエリを指定。
* order ： ソート条件。
* limit ： 検索結果の取得数。-1は無制限。デフォルトは100。
* projection ： 検索結果で取得するフィールドを制限する。

検索の詳細はマニュアルを参照してください。  
[JSONオブジェクトストレージ](https://nec-baas.github.io/baas-manual/latest/developer/ja/developer/functions/object_storage.html)
⇒ オブジェクトの検索(クエリ)

In [ ]:
import necbaas as baas


# 初期化
service = baas.Service(config)

# サーバ証明書検証をスキップする場合のみ False に設定
service.verify_server_cert = False

# 検索対象のオブジェクトバケット
bucket = baas.ObjectBucket(service, "Sample_WeatherData")

# オブジェクト検索実行
results = bucket.query(
    where={"dateTime": {"$gt": "2016-08-01 00:00", "$lte": "2016-09-01 00:00"}},
    order="dateTime",
    limit=-1,
    projection={"ACL": 0, "etag": 0, "createdAt": 0, "updatedAt": 0},
)

# 検索結果の数と先頭のオブジェクトを表示 (for Debug)
print("num of objects:", len(results))
results[0]


necbaasに依存するコードはここまでです。

取得したデータの解析や描画は任意の方法で行ってください。  
本サンプルでは pandas の DataFrame にデータを変換し、グラフ描画します。

## DataFrame データ変換

pandas.DataFrame にデータを変換します。

In [ ]:
import pandas as pd
import dateutil.parser as parser
import copy

def conv(d):
    d = copy.deepcopy(d)
    d.pop("_id")
    d["dateTime"] = parser.parse(d["dateTime"])
    d["windSpeed"] = d["wind"]["speed"]
    d["windDirection"] = d["wind"]["direction"]
    d.pop("wind")
    return d

df = pd.DataFrame(list(map(conv, results)))
df

## グラフ表示

DataFrame.plot() を使用してグラフを表示します。

In [ ]:
df.plot(x="dateTime", y=["airTemp", "rainfall", "windSpeed"], figsize=(16, 8))
